In [1]:
# import internal .py modules
import file_path_management as fpath
import public_library as plib

2023-08-05 21:38:59 GM01X680 metapub.config[76031] WARNING NCBI_API_KEY was not set.


In [2]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re

In [3]:
def www_frontiersin_org(url):
    content = {}
    return content



In [4]:
def extract_info_from_webpage(url, source):
    if source == "www.frontiersin.org":
        url = plib.get_final_redirected_url(url)
        content = www_frontiersin_org(url)
    elif source == "":
        pass
    else:
        pass
# --------------------start of test code--------------------
# url = "https://www.tandfonline.com/doi/abs/10.1080/01616412.1985.11739692"
# source = "www.tandfonline.com"
# content = extract_info_from_webpage(url, source)
# print(content)
# ---------------------end of test code---------------------

In [5]:
# get doi from url
def url2doi(url, source):
    url = str(url).strip()
    # {'www.frontiersin.org', 'www.elibrary.ru', 'orca.cardiff.ac.uk', 'www.jneurosci.org', 
    # 'europepmc.org', 'www.theses.fr', 'www.biorxiv.org', 'submissions.mirasmart.com', 
    # 'royalsocietypublishing.org', 'www.science.org', 'thejns.org', 
    # 'escholarship.mcgill.ca', 'www.cambridge.org', 'movementdisorders.onlinelibrary.wiley.com', 
    # 'www.ahajournals.org', 'books.google.de', 'www.mdpi.com', 'www.sciencedirect.com', 
    # 'ieeexplore.ieee.org', 'academic.oup.com', 'www.pnas.org', 'physoc.onlinelibrary.wiley.com', 
    # 'www.jstage.jst.go.jp', 'wakespace.lib.wfu.edu', 'elibrary.ru', 'www.cabdirect.org', 
    # 'www.tandfonline.com', 'www.jpn.ca', 'jpet.aspetjournals.org', 'onlinelibrary.wiley.com', 
    # 'open.bu.edu', 'tbiomed.biomedcentral.com', 'www.liebertpub.com', 'journals.lww.com', 
    # 'agro.icm.edu.pl', 'ekja.org', 'analyticalsciencejournals.onlinelibrary.wiley.com', 
    # 'n.neurology.org', 'pubs.asahq.org', 'journals.sagepub.com', 'neuro.psychiatryonline.org', 
    # 'karger.com', 'nyaspubs.onlinelibrary.wiley.com', 'pure.mpg.de', 'elifesciences.org', 
    # 'link.springer.com', 'psycnet.apa.org', 'jnnp.bmj.com', 'www.degruyter.com', 'ajp.psychiatryonline.org', 
    # 'journals.physiology.org', 'www.nature.com', 'www.jstor.org', 'var.scholarpedia.org', 'www.eneuro.org', 
    # 'journals.plos.org', 'www.cell.com', 'www.ncbi.nlm.nih.gov', 'www.taylorfrancis.com', 
    # 'bmcneurosci.biomedcentral.com', nan, 'jamanetwork.com'}

    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Firefox(options, service=Service(GeckoDriverManager().install()))
            driver.get(url)

            # WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//p[text()='Consent']"))).click()

            # try:
            #     WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#formInput"))).send_keys(str(doi).strip())
            # except TimeoutException:
            #     print("Waiting for clicking consent timeout")
            # try:
            #     # WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[2]/form/button"))).click()
            #     WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//button[text()="Get PMID"]'))).click()
            # except TimeoutException:
            #     print("Waiting for clicking button timeout")
            
            error_label = 1
        except:
            print("URL to DOI transformation failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0

    try:
        my_elem = driver.find_elements(By.XPATH, "//*[contains(text(), 'doi.org') or contains(text(), 'DOI')]")
        doi = ""
        doi = str(my_elem.find_element(By.XPATH, "i").get_attribute("innerHTML"))
    except:
        doi = np.nan
    driver.quit()
    return doi
# --------------------start of test code--------------------
# url = "https://www.tandfonline.com/doi/abs/10.1080/01616412.1985.11739692"
# doi = url2doi(url)
# print(doi)
# ---------------------end of test code---------------------